# Relion: Power Spectrum preparation for helixplorer
_Adapted by Alex Golubev. CSSB, Landau Group, 2024. The notebook is based on the course materials from Leandro Estrozi.

## To run the notebook copy it in a folder with your Relion project:
preload the pyem module: module load relion/pyem

jupyter notebook --port=$(id -u) --no-browser your_file_name.ipynb
then copy the URL and open it in browser.

## Description of the pipeline
Because we want to avoid to look at a power spectrum (PS) of 2D class-averages, which can have artifacts such as left/right asymmetry, we will calculate a sum of the PS of segments belonging to classes of interest.
Optionally, after the sum of PS of segments belonging to different classes have been calculated, it will be possible to group and align several sums of PS to increase signal.

1. First, set class and job parameters and create a star file with the segments corresponding to the class of interest:

In [ ]:
class_num = 10
job_num = 'job000'

In [ ]:
!module load bsoft
!module load relion

In [ ]:
!relion_star_handler --i Class2D/{job_num}/run_it016_data.star --o Class2D/{job_num}/run_class{class_num}.star --select rlnClassNumber --minval {class_num} --maxval {class_num}

2. Then, we use the in-plane angular information present in the data.star after 2D classification in order to create a stack of aligned segments:

In [ ]:
!relion_stack_create --i Class2D/{job_num}/run_class{class_num}.star --o Class2D/{job_num}/run_class{class_num}_stack --apply_transformation

3. The segments are then padded in a larger box size for finer Fourier sampling. The size after padding depends on your initial box size, we typically use ~twice the box size in pixels. If your original segments are already very large, and a too large box size give you memory problems for next steps, restrict the padded size to the maximum you can.

In [ ]:
!relion_image_handler --i Class2D/{job_num}/run_class{class_num}_stack.mrcs --o Class2D/{job_num}/run_class{class_num}_stack_PAD800.mrcs --new_box 800

4. Now, we calculate the PS (logarithm) for each aligned and padded segment using bshow software (from bsoft). 
If the next cell doesn't work, just open the bshow manualy and find your file

In [ ]:
%%bash -s "$job_num" "$class_num"
module load bsoft
bshow Class2D/${1}/run_class${2}_stack_PAD800.mrcs 

Opening the file could take some time (~1 min).

Use Image => Power spectrum (ONLY use the logarithm option, none of the others). 
Calculation will take a little while depending on how many segments you have per class.  


Use File => Save as… run_class{class}_stack_PAD800_PS.mrcs. This will create a stack of PS.

5. You can then simply sum the aligned PS using :

In [ ]:
!relion_image_handler --i Class2D/{job_num}/run_class{class_num}_stack_PAD800_PS.mrcs --o Class2D/{job_num}/run_class{class_num}_stack_PAD800_PS_average.mrc --average 

6. Look at the PS average: if you see signal only in the low resolution part of the PS, you may want to create a cropped version of the PS, for easier use in helixplorer. If you do not need to crop the PS, ignore the –new_box option and use this command only to convert in .png format. If you decide to crop the PS, note the new pixel size of your final PS sum !

In [ ]:
!relion_image_handler --i Class2D/{job_num}/run_class{class_num}_stack_PAD800_PS_average.mrc --o Class2D/{job_num}/run_class{class_num}_stack_PAD800_PS_average_crop.png --new_box 400